In [37]:
#import required libraries
from datasets import load_dataset
from datasets import load_dataset_builder

import datasets
import numpy as np
import pandas as pd

In [38]:
ds_builder = load_dataset_builder('Falah/Alzheimer_MRI')
print(f"Dataset Name: {ds_builder.info.dataset_name}")
labels = ds_builder.info.features["label"].names
print(f"Class Labels: {labels}")

Dataset Name: alzheimer_mri
Class Labels: ['Mild_Demented', 'Moderate_Demented', 'Non_Demented', 'Very_Mild_Demented']


In [40]:
alzheimer = load_dataset('Falah/Alzheimer_MRI', split='train')
# test_dataset = load_dataset('Falah/Alzheimer_MRI', split="test")
# alzheimer = datasets.DatasetDict({"train":train_dataset,"test":test_dataset})

alzheimer = alzheimer.train_test_split(test_size=0.2)

In [41]:
import numpy as np
imgdata = alzheimer["train"][0]
img = imgdata['image']
imgnp = np.array(img)
print(imgnp.shape)

(128, 128)


In [42]:
#To get the labels from the training set:
labels = alzheimer["train"].features['label'].names #--> returns unique labels. 
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    #print(f'id: {i}, label:{label}')
    label2id[label] = str(i)
    id2label[str(i)] = label

In [43]:
#Print label ids to a label names:
print(f' Label-0 -- > {id2label[str(0)]} \n Label-1 -- > {id2label[str(1)]} \n Label-2 -- > {id2label[str(2)]} \n Label-3 -- > {id2label[str(3)]}')

 Label-0 -- > Mild_Demented 
 Label-1 -- > Moderate_Demented 
 Label-2 -- > Non_Demented 
 Label-3 -- > Very_Mild_Demented


In [44]:
#Next step is to load a ViT image processor to process the image into a tensor
from transformers import AutoImageProcessor
#checkpoint = 'google/vit-base-patch16-224-in21k' #works 
#checkpoint = 'google/vit-hybrid-base-bit-384' --> this can also be tried.
#checkpoint = 'microsoft/swin-base-patch4-window7-224-in22k' 
#checkpoint = 'microsoft/beit-base-patch16-384'  #cuda out of memory
#checkpoint = 'microsoft/swin-base-patch4-window7-224' #works
#checkpoint = 'facebook/deit-base-distilled-patch16-224'#error in training (labels not required)
#checkpoint = 'facebook/deit-base-patch16-224' #works
checkpoint = 'microsoft/beit-base-patch16-224-pt22k-ft22k'
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [45]:
s=checkpoint.split('/')
dest_dir = "models/" + checkpoint.split('/')[1]
dest_dir

'models/beit-base-patch16-224-pt22k-ft22k'

In [46]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)

size = (image_processor.size["shortest_edge"]
        if "shortest_edge" in image_processor.size
        else (image_processor.size["height"], image_processor.size["width"])
)

_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

print(_transforms)


Compose(
    RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear, antialias=warn)
    ToTensor()
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
)


In [47]:
#Create a preprocessing function to apply the transforms and return the pixel_values - the
# inputs to the model -of the image:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples


In [48]:
alzheimer = alzheimer.with_transform(transforms)
alzheimer

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 4096
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 1024
    })
})

In [35]:
#alzheimer['train'][:]['label']

alzheimer['train']['pixel_values']

In [49]:
import numpy as np
d=alzheimer['train'][0]
img = d['pixel_values']
img = np.array(img)
print(img.shape)

(3, 224, 224)


In [50]:
# Now create a batch of examples using DefaultDataCollator. Gap: (Now is this DataLoader of Pytorch?)
from transformers import DefaultDataCollator
data_collator = DefaultDataCollator()

In [51]:
import evaluate
accuracy = evaluate.load("accuracy")

In [52]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions,axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [53]:
from transformers import Trainer, AutoModelForImageClassification, TrainingArguments

model = AutoModelForImageClassification.from_pretrained(checkpoint,
                                                        num_labels=len(labels),
                                                        id2label = id2label,
                                                        label2id = label2id,
                                                        ignore_mismatched_sizes=True
                                                        )


Some weights of BeitForImageClassification were not initialized from the model checkpoint at microsoft/beit-base-patch16-224-pt22k-ft22k and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([21841, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([21841]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [54]:
training_args = TrainingArguments(
    output_dir="models/" + checkpoint.split('/')[1],
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=50,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

In [55]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=alzheimer["train"],
    eval_dataset=alzheimer["test"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

In [56]:
trainer.train()

# trainer.evaluate() — The dataset to use for evaluation.
#  If it is a Dataset, columns not accepted by the model.forward() method are automatically removed. 
# If it is a dictionary, it will evaluate on each dataset prepending the dictionary key to the metric name.

  0%|          | 0/3200 [00:00<?, ?it/s]

{'loss': 1.3514, 'learning_rate': 1.5625e-06, 'epoch': 0.16}
{'loss': 1.1677, 'learning_rate': 3.125e-06, 'epoch': 0.31}
{'loss': 1.0548, 'learning_rate': 4.6875000000000004e-06, 'epoch': 0.47}
{'loss': 1.0066, 'learning_rate': 6.25e-06, 'epoch': 0.62}
{'loss': 0.9683, 'learning_rate': 7.8125e-06, 'epoch': 0.78}
{'loss': 0.9493, 'learning_rate': 9.375000000000001e-06, 'epoch': 0.94}


  0%|          | 0/64 [00:00<?, ?it/s]

{'eval_loss': 0.9605082869529724, 'eval_accuracy': 0.564453125, 'eval_runtime': 10.5124, 'eval_samples_per_second': 97.409, 'eval_steps_per_second': 6.088, 'epoch': 1.0}
{'loss': 0.9266, 'learning_rate': 1.09375e-05, 'epoch': 1.09}
{'loss': 0.8731, 'learning_rate': 1.25e-05, 'epoch': 1.25}
{'loss': 0.9275, 'learning_rate': 1.4062500000000001e-05, 'epoch': 1.41}
{'loss': 0.9191, 'learning_rate': 1.5625e-05, 'epoch': 1.56}
{'loss': 0.8515, 'learning_rate': 1.71875e-05, 'epoch': 1.72}
{'loss': 0.8779, 'learning_rate': 1.8750000000000002e-05, 'epoch': 1.88}


  0%|          | 0/64 [00:00<?, ?it/s]

...

  0%|          | 0/64 [00:00<?, ?it/s]

{'eval_loss': 0.15349119901657104, 'eval_accuracy': 0.955078125, 'eval_runtime': 10.7715, 'eval_samples_per_second': 95.065, 'eval_steps_per_second': 5.942, 'epoch': 49.0}
{'loss': 0.057, 'learning_rate': 1.0416666666666667e-06, 'epoch': 49.06}
{'loss': 0.0512, 'learning_rate': 8.680555555555556e-07, 'epoch': 49.22}
{'loss': 0.091, 'learning_rate': 6.944444444444445e-07, 'epoch': 49.38}
{'loss': 0.0842, 'learning_rate': 5.208333333333334e-07, 'epoch': 49.53}
{'loss': 0.0851, 'learning_rate': 3.4722222222222224e-07, 'epoch': 49.69}
{'loss': 0.0722, 'learning_rate': 1.7361111111111112e-07, 'epoch': 49.84}
{'loss': 0.0725, 'learning_rate': 0.0, 'epoch': 50.0}


  0%|          | 0/64 [00:00<?, ?it/s]

{'eval_loss': 0.1159391850233078, 'eval_accuracy': 0.9638671875, 'eval_runtime': 10.8811, 'eval_samples_per_second': 94.108, 'eval_steps_per_second': 5.882, 'epoch': 50.0}
{'train_runtime': 5829.6633, 'train_samples_per_second': 35.131, 'train_steps_per_second': 0.549, 'train_loss': 0.27567509358748793, 'epoch': 50.0}


TrainOutput(global_step=3200, training_loss=0.27567509358748793, metrics={'train_runtime': 5829.6633, 'train_samples_per_second': 35.131, 'train_steps_per_second': 0.549, 'train_loss': 0.27567509358748793, 'epoch': 50.0})

In [35]:
#Read the Test Data and do the same preprocessing.
testdata = load_dataset('Falah/Alzheimer_MRI', split='test')
testdata = testdata.with_transform(transforms)

In [36]:
#Method-2: using evaluate (Gives overall accuracy)
results=trainer.evaluate(testdata)
## produces the below output ### 
{'eval_loss': 2.117784023284912,
 'eval_accuracy': 0.831,
 'eval_runtime': 310.6302,
 'eval_samples_per_second': 3.219,
 'eval_steps_per_second': 0.203,
 'epoch': 1.98}
print(results)
# df=pd.DataFrame(results)
# df.to_csv('/Results/evaluation_results.csv',index=False)

  0%|          | 0/80 [00:00<?, ?it/s]

{'eval_loss': 0.2076101303100586, 'eval_accuracy': 0.93125, 'eval_runtime': 13.0413, 'eval_samples_per_second': 98.15, 'eval_steps_per_second': 6.134, 'epoch': 50.0}


In [22]:
results

{'eval_loss': 0.3861716091632843,
 'eval_accuracy': 0.859375,
 'eval_runtime': 12.5942,
 'eval_samples_per_second': 101.634,
 'eval_steps_per_second': 6.352,
 'epoch': 20.0}

In [ ]:
#use of pipeline to get the predictions --> Way Forward. 


In [18]:
history

TrainOutput(global_step=1280, training_loss=0.5180638987571001, metrics={'train_runtime': 2372.3217, 'train_samples_per_second': 34.532, 'train_steps_per_second': 0.54, 'train_loss': 0.5180638987571001, 'epoch': 20.0})

It depends on what you’d like to do, trainer.evaluate() will predict + compute metrics on your test set and trainer.predict() will only predict labels on your test set. However in case the test set also contains ground-truth labels, the latter will also compute metrics.

In [69]:
#Method-3: Using predict (Gives predicted label, probability values for each sample along with metrics)
results_prediction = trainer.predict(testdata)

  0%|          | 0/80 [00:00<?, ?it/s]

In [113]:
results_prediction

PredictionOutput(predictions=array([[-0.5591147 , -2.0389035 ,  1.242404  ,  0.98351747],
       [-1.0933479 , -1.7672849 ,  1.4555196 ,  0.76520914],
       [-1.2960124 , -1.39749   ,  2.033903  , -0.04442274],
       ...,
       [-1.2275118 , -1.7329051 ,  1.8181549 ,  0.63654417],
       [-1.1876274 , -1.6070359 ,  1.8790383 ,  0.40170136],
       [ 0.2324425 , -1.971694  ,  0.30193636,  1.0467229 ]],
      dtype=float32), label_ids=array([3, 0, 2, ..., 3, 2, 3], dtype=int64), metrics={'test_loss': 0.8645501136779785, 'test_accuracy': 0.584375, 'test_runtime': 12.1436, 'test_samples_per_second': 105.405, 'test_steps_per_second': 6.588})

In [85]:
prediction=results_prediction[1] # predicted labels
actual = np.array(testdata[:]['label']) #ground-truth labels
print(prediction.shape)
print(actual.shape)

(1280,)
(1280,)


In [116]:
proba = results_prediction[0]
#proba
#print(proba.shape)
prediction1=np.argmax(proba, axis=1)
#print(actual1.shape)
#print(proba[96])
#print(prediction1[96])

print(prediction[1:10])
print(prediction1[1:10])


[0 2 3 0 3 2 2 2 3]
[2 2 3 3 3 3 3 3 3]


In [108]:
print(proba[56])

print(np.argmax(proba[56]))


[ 0.73476785 -1.6395384  -0.03571419  0.6846446 ]
0
